<img src='https://neurohive.io/wp-content/uploads/2019/07/Screenshot-from-2019-07-25-01-30-57.png'>
<p>
<p>
<h1><center>Lyft Motion Prediction for Autonomous Vehicles</center><h1>
<h3><center>&#128663;Build motion prediction models for self-driving vehicles&#128663;</center><h3>

# Table of contents🗃<a id='0.1'></a>

- [0. Introduction](#0)

- [1. Import Packages](#1)

- [2. Utility Functions](#2)
- [3. Basic EDA](#3)
    - [3.1 Data Formats](#3-1)
        - [3.1.1 Introduction](#3-1-1)
        - [3.1.2 Zarr Format](#3-1-2)
        - [3.1.3 Train, Validation And Test Zarr](#3-1-3)
    - [3.2 Checking Configuration Fields](#3-2)
    - [3.3 Loading Data](#3-3)
    - [3.4 Data Overview](#3-4)
        - [3.4.1 Agents](#3-4-1)
        - [3.4.2 Scenes](#3-4-2)
        - [3.4.3 Frames](#3-4-3)
    - [3.5 Dataset Package](#3-5)
        - [3.5.1 ChunckedDataset](#3-5-1)
    - [3.6 Visualize Autonomous Vehicle](#3-6)  
        - [3.6.1 Visualizing Various Rasterizer Objects](#3-6-1)  
        - [3.6.2 Visualize Trajectory: Semantic View](#3-6-2)
        - [3.6.3 Visualize Trajectory: Satellite View](#3-6-3)
        - [3.6.4 Visualize Agent](#3-6-4)
        - [3.6.5 Visualize Individual Scene: Semantic](#3-6-5)
        - [3.6.6 Visualize Individual Scene: Satellite](#3-6-6)
        
- [4. Pytorch Baseline](#4)
    - [4.1 Configuration](#4-1)
    - [4.2 Loading Training Data](#4-2)
    - [4.3 Training DataLoader](#4-3)
    - [4.4 Model: resnet50 (Pytorch)](#4-4)
    - [4.5 Compilation](#4-5)
    - [4.6 Training](#4-6)
    - [4.7 Saving Model](#4-7)
    
- [5. Prediction and Results](#5)
    - [5.1 Test DataLoader](#5-1)
    - [5.2 Getting Predictions](#5-2)
    - [5.2 Submission](#5-3)
    
- [6. Reference](#6)

# 0. <a id='0'>Introduction📃</a>

[Table of Contents](#0.1)

This competition is hosted by ridesharing company [Lyft](https://www.lyft.com/) which started [Level 5](https://self-driving.lyft.com/level5/), self-driving division to tackle the challenges in the field of self-driving cars. 

In this competition our task is to build motion prediction models for self-driving vehicles. Model which can predict the movement of traffic agents around the Autonomous Vehicles such as cars, cyclists, and pedestrians etc. We are required to predict how these different agents move in Autonomous Vehicles's environment.

## What is Autonomous Vehicle?

An autonomous vehicle, or a driverless vehicle, is one that is able to operate itself and perform necessary functions without any human intervention, through ability to sense its surroundings.

An autonomous vehicle utilises a fully automated driving system in order to allow the vehicle to respond to external conditions that a human driver would manage.

## Competition Data

This [dataset](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/data) is the largest collection of the traffic agent motion data. This dataset includes the logs of movement of cars, cyclists, pedestrians, and other traffic agents encountered by Lyft's autonomous fleet. These logs come from processing raw lidar, camera, and radar data through our team’s perception systems and are ideal for training motion prediction models. The dataset includes:

* 1000+ hours of traffic agent movement
* 16k miles of data from 23 vehicle
* 15k semantic map annotations

Here is the reserach paper of [Prediction Dataset](https://paperswithcode.com/paper/one-thousand-and-one-hours-self-driving).

The dataset consists of 170,000 scenes capturing the environment around the autonomous vehicle. Each scene encodes the state of the vehicle’s surroundings at a given point in time. [source](https://self-driving.lyft.com/level5/prediction/)

<div style="clear:both;display:table">
<img src="https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_lrg_redux.gif" style="width:45%;float:left"/>
<img src="https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_2-1.png" style="width:45%;float:left"/>
</div>
<br/>

This baseline solution is trained on over 2 million samples from the agent locations contained within the dataset. The model predicts a single agent at a time. First, a raster generates a bird’s eye view (BEV) top-down raster, which encodes all agents and the map. The network infers the future coordinates of the agent-based upon this raster.

<img src="https://self-driving.lyft.com/wp-content/uploads/2020/06/diagram-prediction-1.jpg" style="width:70%"/>
<br/>
    
## What we are predicting?

Our task in the competition is to predict the motion of external objects such as cars, cyclist, pedestrains etc in order to assist the self-driving car. We have to predict the location of objects agents in the next 50 frames.

## Evaluation Metric: Negative log-likelihood

We calculate the negative log-likelihood of the ground truth data given the multi-modal predictions. You can get more information [here](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/overview/evaluation).


![](https://camo.githubusercontent.com/b3634eea5be5501318957e21086781666018efa1/68747470733a2f2f6c617465782e636f6465636f67732e636f6d2f6769662e6c617465783f25354362675f77686974652532302535436c617267652532304c2532302533442532302d2532302535436c6f6725323070253238785f253742312532432532302535436c646f747325324325323054253744253243253230795f253742312532432532302535436c646f74732532432532305425374425374363253545253742312532432532302535436c646f74732532432532304b253744253243253230253543626172253742782537445f253742312532432532302535436c646f747325324325323054253744253545253742312532432532302535436c646f74732532432532304b253744253243253230253543626172253742792537445f253742312532432532302535436c646f747325324325323054253744253545253742312532432532302535436c646f74732532432532304b253744253239)
![](https://camo.githubusercontent.com/8048a110a20827715a17eb76f8039302a576d503/68747470733a2f2f6c617465782e636f6465636f67732e636f6d2f6769662e6c617465783f25354362675f77686974652532302535436c617267652532302533442532302d2532302535436c6f6725323025354373756d5f6b253230652535452537422535436c6f67253238632535456b2532392532302b25323025354373756d5f742532302535436c6f672532302535436d61746863616c2537424e253744253238785f74253743253543626172253742782537445f742535456b2532432532302535437369676d61253344312532392532302535436d61746863616c2537424e253744253238795f74253743253543626172253742792537445f742535456b2532432532302535437369676d6125334431253239253744)
![](https://camo.githubusercontent.com/9ba94f5c0c40666d66b93fba994cc5f7623ebd98/68747470733a2f2f6c617465782e636f6465636f67732e636f6d2f6769662e6c617465783f25354362675f77686974652532302535436c617267652532302533442532302d2532302535436c6f6725323025354373756d5f6b253230652535452537422535436c6f67253238632535456b2532392532302d25354366726163253742312537442537423225374425323025354373756d5f74253230253238253543626172253742782537445f742535456b2532302d253230785f74253239253545322532302b253230253238253543626172253742792537445f742535456b2532302d253230795f7425323925354532253744)

## Lyft's Autonomous Vehicle (AV)

**Lyft's** AV Introducton video.  

In [ ]:
from IPython.display import HTML

HTML('<center><iframe  width="850" height="450" src="https://www.youtube.com/embed/K0H43N-Hx7w" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

# 1. <a id='1'>Import Packages📚</a>

[Table of Contents](#0.1)

We are required to use [L5Kit toolkit](https://github.com/lyft/l5kit) provided by the competition host to prepare/preprocess data, trian and evaluate the model. Please add this [utility script](https://www.kaggle.com/pestipeti/lyft-l5kit-unofficial-fix) provided by [Peter's](https://www.kaggle.com/pestipeti) first by clicking on the [+ Add data]() section inside your notebook before starting. The [L5Kit toolkit](https://github.com/lyft/l5kit) has some issue right now. You can check [this](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/discussion/177125) discussion here for more information.  

## L5Kit is a library which lets you:

* Load driving scenes from zarr files
* Read semantic maps
* Read aerial maps
* Create birds-eye-view (BEV) images which represent a scene around an AV or another vehicle
* Sample data
* Train neural networks
* Visualize results

In [ ]:
# import packages
import os, gc
import zarr
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from typing import Dict
from collections import Counter
from prettytable import PrettyTable

#level5 toolkit
from l5kit.data import PERCEPTION_LABELS
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.data import ChunkedDataset, LocalDataManager

# level5 toolkit 
from l5kit.configs import load_config_data
from l5kit.geometry import transform_points
from l5kit.rasterization import build_rasterizer
from l5kit.visualization import draw_trajectory, draw_reference_trajectory, TARGET_POINTS_COLOR
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from colorama import Fore, Back, Style

# deep learning
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18, resnet50, resnet34

# check files in directory
print((os.listdir('../input/lyft-motion-prediction-autonomous-vehicles/')))

plt.rc('animation', html='jshtml')

%matplotlib inline

# 2. <a id='2'>Utility Functions🔨</a>

Thanks to [Trigram](https://www.kaggle.com/nxrprime) for providing this function.

In [ ]:
# animation for scene
def animate_solution(images):

    def animate(i):
        im.set_data(images[i])
 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    
    return animation.FuncAnimation(fig, animate, frames=len(images), interval=80)

# 3. <a id='3'>EDA🔍</a>

[Table of Contents](#0.1)

The data is huge around 22 GB. We will use Lyft's [L5Kit](https://github.com/lyft/l5kit) to process our data (loading and visualization) and training. 

## 3.1 <a id='3-1'>Data Formats</a>

We need to be familiar with the data we are using. Let's dive in.. 

### 3.1.1 <a id='3-1-1'>Introduction</a>

The L5Kit toolkit uses data format that consists of a set of numpy structured arrays. **[Structured arrays](https://numpy.org/doc/stable/user/basics.rec.html)** are ndarrays whose datatype is a composition of simpler datatypes organized as a sequence of named fields. The structured array can store various types of features. Structured arrays are stored in memory in an interleaved format, this means that one "row" or "sample" is grouped together in memory. Let us take an example to understand this. 

In [ ]:
import numpy as np
my_arr = np.zeros(3, dtype=[("color", (np.uint8, 3)), ("label", np.bool)])

print(my_arr[0])

let's add some data in this structured array.

In [ ]:
my_arr[0]["color"] = [0, 218, 130]
my_arr[0]["label"] = True
my_arr[1]["color"] = [245, 59, 255]
my_arr[1]["label"] = True
my_arr[1]["color"] = [7, 6, 97]
my_arr[1]["label"] = True

print(my_arr)

As we can see structured arrays allow us to mix different data types into a single array. We will develop more intuition ahead.

### 3.1.2 <a id='3-1-2'>Zarr Format</a>

[Table of Contents](#0.1)

The L5Kit uses zarr format to store and read these structured numpy arrays. The data is available in .zarr file format which can be easily load using the [L5Kit](https://github.com/lyft/l5kit). Most of the traditional numpy operations can be handled using .zarr files. The zarr files are flat, compact, and highly performant for loading. Each of the .zarr file contains - 

   * scenes: driving episodes acquired from a given vehicle.
   * frames: snapshots in time of the pose of the vehicle. A frame is a snapshot in time which consists of ego pose, time, and multiple agent states.
   * agents: a generic entity captured by the vehicle's sensors. Note that only 4 of the 17 possible agent label_probabilities are present in this dataset. Each agent state describes the position, orientation, bounds, and type.
   * agents_mask: a mask that (for train and validation) masks out objects that aren't useful for training. In test, the mask (provided in files as mask.npz) masks out any test object for which predictions are NOT required.
   * traffic_light_faces: traffic light information.


### 3.1.3 <a id='3-1-3'>Train, Validation And Test Zarr</a>

[Table of Contents](#0.1)

In [ ]:
train = zarr.open("../input/lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr")
validation = zarr.open("../input/lyft-motion-prediction-autonomous-vehicles/scenes/validate.zarr")
test = zarr.open("../input/lyft-motion-prediction-autonomous-vehicles/scenes/test.zarr/")
train.info

We can see that our train.zarr file has set of 4 arrays. All this 4 fields are described above in the **Data Format** section above. Let us check these fields. We will check 1 observation from each.

In [ ]:
print(f'We have {len(train.agents)} agents, {len(train.scenes)} scenes, {len(train.frames)} frames and {len(train.traffic_light_faces)} traffic light faces in train.zarr.')
print(f'We have {len(validation.agents)} agents, {len(validation.scenes)} scenes, {len(validation.frames)} frames and {len(validation.traffic_light_faces)} traffic light faces in validation.zarr.')
print(f'We have {len(test.agents)} agents, {len(test.scenes)} scenes, {len(test.frames)} frames and {len(test.traffic_light_faces)} traffic light faces in test.zarr.')


The data is expected to live in a folder that can be configured using the L5KIT_DATA_FOLDER env variable. We will now develop some intuition about the data.

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"

# get configuration yaml
cfg = load_config_data("../input/lyft-config-files/visualisation_config.yaml")
print(cfg)

## 3.2 <a id='3-2'>Checking Configuration Fields🗄</a>

[Table of Contents](#0.1)

We will look at this yaml file from an external dataset(provided in L5kit examples). Let's look the **raster_params** filed. It contains information related to the transformation of the 3D world onto image plane. You can also check for various other information.

## [NOTE]:We can make our own configuration file. 

In [ ]:
# Raster Parameters
print(f'current raster_param:\n')
for k,v in cfg["raster_params"].items():
    print(f"{k}:{v}")

## 3.3 <a id='3-3'>Loading Data</a>

[Table of Contents](#0.1)

We're building a LocalDataManager object. This will resolve relative paths from the config using the L5KIT_DATA_FOLDER env variable we have just set. We will work with sample.zarr for developing intuition regarding the data. Please use train.zarr, validate.zarr and test.zarr for actual model training, validation and prediction.

In [ ]:
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

In [ ]:
print(dataset_path)

We are going to load our sample.zarr file. The 2020 Lyft competition dataset is stored in four structured arrays: **scenes, frames, agents and tl_faces**. 

## 3.4 <a id='3-4'>Data Overview</a>

[Table of Contents](#0.1)

We will see each field inside the zarr files. (train, validation and test). We will use **sample.zarr** here. 

### 3.4.1 <a id='3-4-1'>Agents</a>

An agent is an observation by the AV of some other detected object. Each entry describes the object in terms of its attributes such as position and velocity, gives the agent a tracking number to track it over multiple frames (but only within the same scene!) and its most probable label.

In [ ]:
agents = pd.DataFrame.from_records(zarr_dataset.agents, columns = ['centroid', 'extent', 'yaw', 'velocity', 'track_id', 'label_probabilities'])
agents.head()

**📌 Points to note :**
   * **centroid** - position of agent
   * **extent** - agent dimension
   * **yaw** - rotation of an agent with respect to vertical axis. A yaw rotation is a movement around the yaw axis of a rigid body that changes the direction it is pointing, to the left or right of its direction of motion.
   * **velocity** - speed of the agent 
   * **track_id** - unique id to track agent in different frames
   * **label_probabilities** - prabability an agent belong to one of the 17 classes. (We are only given three labels [cyclist, pedestrians and cars])

### Centroid Distribution

We will see the distribution of centroid now. Since **centroid** column consist of list per sample we will make two new columns **centroid_x** and **centroid_y**.

In [ ]:
agents[['centroid_x','centroid_y']] = agents['centroid'].to_list()
agents = agents.drop('centroid', axis=1)
agents_new = agents[["centroid_x", "centroid_y", "extent", "yaw", "velocity", "track_id", "label_probabilities"]]
del agents
agents_new

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
plt.scatter(agents_new['centroid_x'], agents_new['centroid_y'], marker='+')
plt.xlabel('x', fontsize=11); plt.ylabel('y', fontsize=11)
plt.title("Centroids distribution (sample.zarr)")
plt.show()

### Extent Distribution

First we need to make new columns for extent_x, extent_y and extent_z as we have one **extent** column.

In [ ]:
agents_new[['extent_x','extent_y', 'extent_z']] = agents_new['extent'].to_list()
agents_new = agents_new.drop('extent', axis=1)
agents = agents_new[["centroid_x", "centroid_y", 'extent_x', 'extent_y', 'extent_z', "yaw", "velocity", "track_id", "label_probabilities"]]
del agents_new
agents

In [ ]:
sns.axes_style("white")

fig, ax = plt.subplots(1,3,figsize=(16,5))

plt.subplot(1,3,1)
sns.kdeplot(agents['extent_x'], shade=True, color='red');
plt.title("Extent_x distribution")

plt.subplot(1,3,2)
sns.kdeplot(agents['extent_y'], shade=True, color='steelblue');
plt.title("Extent_y distribution")

plt.subplot(1,3,3)
sns.kdeplot(agents['extent_z'], shade=True, color='green');
plt.title("Extent_z distribution")

plt.show();

**📌 Points to note :**
   * We can see extent distributions are **right skewed**.
   * We can see long tails in positive direction.

### Extent Distribution: Scatterplot

In [ ]:
sns.set_style('whitegrid')

fig, ax = plt.subplots(1,3,figsize=(16,5))
plt.subplot(1,3,1)
plt.scatter(agents['extent_x'], agents['extent_y'], marker='*')
plt.xlabel('ex', fontsize=11); plt.ylabel('ey', fontsize=11)
plt.title("Extent: ex-ey")

plt.subplot(1,3,2)
plt.scatter(agents['extent_y'], agents['extent_z'], marker='*', color="red")
plt.xlabel('ey', fontsize=11); plt.ylabel('ez', fontsize=11)
plt.title("Extent: ey-ez")

plt.subplot(1,3,3)
plt.scatter(agents['extent_z'], agents['extent_x'], marker='*', color="green")
plt.xlabel('ez', fontsize=11); plt.ylabel('ex', fontsize=11)
plt.title("Extent: ez-ex")

plt.show();

### Yaw Distribution

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8))
sns.distplot(agents['yaw'])
plt.title("Yaw Distribution")
plt.show()

### Velocity Distribution

In [ ]:
agents[['velocity_x','velocity_y']] = agents['velocity'].to_list()
agents_vel = agents.drop('velocity', axis=1)
agents_v = agents_vel[["centroid_x", "centroid_y", 'extent_x', 'extent_y', 'extent_z', "yaw", "velocity_x", "velocity_y", "track_id", "label_probabilities"]]
del agents
agents_v

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8))

with sns.axes_style("whitegrid"):
    sns.scatterplot(x=agents_v["velocity_x"], y=agents_v["velocity_y"], color='k');
    plt.title('Velocity Distribution')

In [ ]:
agents = zarr_dataset.agents
probabilities = agents["label_probabilities"]
labels_indexes = np.argmax(probabilities, axis=1)
counts = []
for idx_label, label in enumerate(PERCEPTION_LABELS):
    counts.append(np.sum(labels_indexes == idx_label))
    
table = PrettyTable(field_names=["label", "counts"])
for count, label in zip(counts, PERCEPTION_LABELS):
    table.add_row([label, count])
print(table)

As we can see indeed there are only four types of agents provided in the dataset such as Cars, Cyclists, Pedestrians and Unknown. We can see that **Unknown** label is more as compared to other three agent labels. 

In [ ]:
print(f'{Fore.YELLOW}Total number of agents in sample .zarr files is {Style.RESET_ALL}{len(zarr_dataset.agents)}. {Fore.BLUE}\nAfter summing up the elements in count column we can see we have {Style.RESET_ALL}{(1324481 + 519385 + 6688 + 43182)} {Fore.BLUE}agents in total.')

### 3.4.2 <a id='3-4-2'>Scenes</a>
[Table of contents](#0.1)

A scene is identified by the host (i.e. which car was used to collect it) and a start and end time. It consists of multiple frames (=snapshots at discretized time intervals). The scene datatype stores references to its corresponding frames in terms of the start and end index within the frames array (described in dataframe below). The frames in between these indices all correspond to the scene (including start index, excluding end index).

In [ ]:
scenes = pd.DataFrame.from_records(zarr_dataset.scenes, columns = ['frame_index_interval', 'host', 'start_time', 'end_time'])
scenes.head()

**📌 Points to note :**
   * **frame_index_interval** - frame index (including start index, excluding end index)
   * **host** - car used to collect data
   * **start_time** - start time of scene
   * **end_time** - end time of scene

### Frame Index

In [ ]:
scenes[['frame_start_index','frame_end_index']] = scenes['frame_index_interval'].to_list()
scenes_new = scenes.drop('frame_index_interval', axis=1)
scenes_new = scenes_new[["frame_start_index", "frame_end_index", 'host', 'start_time', 'end_time']]
del scenes
scenes_new.head()

In [ ]:
f = plt.figure(figsize=(10, 8))
gs = f.add_gridspec(1, 2)

with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[0,0])
    sns.scatterplot(scenes_new['frame_start_index'], scenes_new['frame_end_index'])
    plt.title('Frame Index Interval Distribution')
    
with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[0,1])
    sns.scatterplot(scenes_new['frame_start_index'], scenes_new['frame_end_index'], hue=scenes_new['host'])
    plt.title('Frame Index Interval Distribution (Grouped per host)')
    
f.tight_layout()

**📌 Points to note :**
   * We can see linear trend here.
   * Both host cars collected data within particular time intervals.

### Host Count

We will now see the host counts (Ego vehicle used to collect the data).

In [ ]:
f = plt.figure(figsize=(10, 8))

with sns.axes_style("white"):
    sns.countplot(scenes_new['host']);
    plt.title("Host Count")

**📌 Points to note :**
   * We have two host cars which were used to collect the data **"host-a013"** and **"host-a101"**.

### 3.4.3 <a id='3-4-3'>Frames</a>
[Table of contents](#0.1)

A frame captures all information that was observed at a time. This includes the following fields as mentioned in dataframe below.

In [ ]:
frames = pd.DataFrame.from_records(zarr_dataset.frames, columns = ['timestamp', 'agent_index_interval', 'traffic_light_faces_index_interval', 'ego_translation','ego_rotation'])
frames.head()

**📌 Points to note :**
   * **timestamp** - frame's timestamp
   * **agent_index_interval** - agents (vehicles, cyclists and pedestrians) that were captured by the ego's sensors
   * **traffic_light_faces_index_interval** - traffic light index
   * **ego_translation** - position of host car.
   * **ego_rotation** - rotation of host car (which is collecting data using ego sensors)

### Ego Translation Distribution

In [ ]:
frames[['ego_translation_x', 'ego_translation_y', 'ego_translation_z']] = frames['ego_translation'].to_list()
frames_new = frames.drop('ego_translation', axis=1)
frames_new = frames_new[['timestamp', 'agent_index_interval', 'traffic_light_faces_index_interval',
                         'ego_translation_x', 'ego_translation_y', 'ego_translation_z', 'ego_rotation']]
del frames
frames_new.head()

In [ ]:
f = plt.figure(figsize=(16, 8))
gs = f.add_gridspec(1, 3)

with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[0,0])
    sns.distplot(frames_new['ego_translation_x'], color='Orange')
    plt.title('Ego Translation Distribution X')
    
with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[0,1])
    sns.distplot(frames_new['ego_translation_y'], color='Red')
    plt.title('Ego Translation Distribution Y')
    
with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[0,2])
    sns.distplot(frames_new['ego_translation_z'], color='Green')
    plt.title('Ego Translation Distribution Z')
    
f.tight_layout()

**📌 Points to note :**
   * We can see the distributions are multi-models here.

### Ego Translation Scatterplot

In [ ]:
f = plt.figure(figsize=(16, 6))
gs = f.add_gridspec(1, 3)

with sns.axes_style("darkgrid"):
    ax = f.add_subplot(gs[0,0])
    plt.scatter(frames_new['ego_translation_x'], frames_new['ego_translation_y'],
                    color='darkkhaki', marker='+')
    plt.title('Ego Translation X-Y')
    plt.xlabel('ego_translation_x')
    plt.ylabel('ego_translation_y')
    
with sns.axes_style("darkgrid"):
    ax = f.add_subplot(gs[0,1])
    plt.scatter(frames_new['ego_translation_y'], frames_new['ego_translation_z'],
                    color='slateblue', marker='*')
    plt.title('Ego Translation Distribution Y-Z')
    plt.xlabel('ego_translation_y')
    plt.ylabel('ego_translation_z')
    
with sns.axes_style("darkgrid"):
    ax = f.add_subplot(gs[0,2])
    plt.scatter(frames_new['ego_translation_z'], frames_new['ego_translation_x'],
                    color='turquoise', marker='^')
    plt.title('Ego Translation Distribution Z-X')
    plt.xlabel('ego_translation_z')
    plt.ylabel('ego_translation_x')
    
f.tight_layout()

### Ego Rotations Distribution

In [ ]:
fig, ax = plt.subplots(3,3,figsize=(16,16))
colors = ['red', 'blue', 'green', 'magenta', 'orange', 'darkblue', 'black', 'cyan', 'darkgreen']
for i in range(0,3):
    for j in range(0,3):
        df = frames_new['ego_rotation'].apply(lambda x: x[i][j])
        plt.subplot(3,3,i * 3 + j + 1)
        sns.distplot(df, hist=False, color = colors[ i * 3 + j  ])
        plt.xlabel(f'r[ {i + 1} ][ {j + 1} ]')
fig.suptitle("Ego rotation angles distribution", size=14)
plt.show()

### 3.4.3 <a id='3-4-3'>Traffic Light Faces</a>
[Table of contents](#0.1)

The traffic light bulbs (red, green, yellow) are refered as **face**.

In [ ]:
traffic_light_faces = pd.DataFrame.from_records(zarr_dataset.tl_faces, columns = ['face_id', 'traffic_light_id', 'traffic_light_face_status'])
traffic_light_faces.head()

   * **face_id** - unique id for traffic light bulb
   * **traffic_light_id** - traffic light status
   * **traffic_light_face_status** - out of red/green/yellow which face is active/unactive/unknown

## 3.5 <a id='3-5'>Dataset Package</a>

[Table of Contents](#0.1)

The dataset package, for example, already implements PyTorch ready datasets, so you can hit the ground running and start coding immediately. There are two classes in the dataset package. We will be using below two datasets classes to generate inputs and targets.

   * EgoDataset: this dataset iterates over the AV annotations
   * AgentDataset: this dataset iterates over other agents annotations
   
Both of them can be iterated and return multi-channel images from the rasterizer along with future trajectories offsets and other information. We will see ahead about **rasterizer and trajectories** soon.

We need **cfg**, **Zarr-Dataset(ChunkedDataset)** and **rasterizer** object to instantiate these.

### 3.5.1 <a name='3-5-1'>ChunckedDataset</a>

[Table of Contents](#0.1)

The **[ChunckedDataset](#0.2)**(click) class as you can see above returned four structured arrays, scenes, frames, agents and tl_faces, all are described above in detail. Both the **EgoDataset and AgentDataset** are using **zarr dataset** object which is made using **[ChunckedDataset](#0.2)**. 


## 3.6 <a id='3-6'>Visualize Autonomous Vehicle🚙</a>

[Table of Contents](#0.1)

Now we will look into the visualisation utility of [L5Kit Toolkit](https://github.com/lyft/l5kit). There are two core packages for visualisation:

* **rasterization** - contains classes for getting visual data as multi-channel tensors and turning them into interpretable RGB images. We will talk about these classes in detail shortly. Each class inside this [rasterization](https://github.com/lyft/l5kit/tree/master/l5kit/l5kit/rasterization) package conatin has at least a **rasterize** method to get the tensor and a **to_rgb** method to convert it into an image. 

    * BoxRasterizer: this object renders agents (e.g. vehicles or pedestrians) as oriented 2D boxes.
    * StubRasterizer: this object doesn't do anything. It return all black image and can be used for testing.
    * SatelliteRasterizer: this object renders an oriented crop from a satellite map.
    * SatBoxRasterizer: this object combine a **Satellite** and a **Box Rasterizers** into a single class. You can visualize agents (e.g. vehicles or pedestrians) as oriented 2D boxes in a satellite image.
    * SemanticRasterizer: this object renders semantic map which contains lane & crosswalk information.
    * SemBoxRasterizer: this object combine a **Semantic Map** and a **Box Rasterizers** into a single class. You can visualize agents (e.g. vehicles or pedestrians) as oriented 2D boxes in a semantic image.
    
To instantiate each of these object we will use **build_rasterizer** method.
   
   
* **visualization** - contains utilities to draw additional information (e.g. trajectories) onto RGB images. These utilities are commonly used after a **to_rgb** call to add other information to the final visualisation. Following utilities are available.
   
   * draw_arrowed_line: Draw a single arrowed line in an RGB image.
   * draw_trajectory: Draw a trajectory on oriented arrow onto an RGB image.
   * draw_reference_trajectory: Draw a trajectory (as points) onto the image.

### **3.6.1<a id='3-6-1'> Visualizing Various Rasterizer Objects**</a>

We will visualize different raster objects. [credits](https://www.kaggle.com/corochann/lyft-deep-into-the-l5kit-library#1.-Understanding-Rasterizer-class)

[Table of Contents](#0.1)

In [ ]:
sns.set_style({'axes.grid': False})

def visualize_rgb_image(dataset, index, title="", ax=None):
    """Visualizes Rasterizer's RGB image"""
    data = dataset[index]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)

    if ax is None:
        fig, ax = plt.subplots()
    if title:
        ax.set_title(title)
    ax.imshow(im[::-1])
# Prepare all rasterizer and EgoDataset for each rasterizer
rasterizer_dict = {}
dataset_dict = {}

rasterizer_type_list = ["py_satellite", "satellite_debug", "py_semantic", "semantic_debug", "box_debug", "stub_debug"]

for i, key in enumerate(rasterizer_type_list):
    # print("key", key)
    cfg["raster_params"]["map_type"] = key
    rasterizer_dict[key] = build_rasterizer(cfg, dm)
    dataset_dict[key] = EgoDataset(cfg, zarr_dataset, rasterizer_dict[key])
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
for i, key in enumerate(["stub_debug", "satellite_debug", "semantic_debug", "box_debug", "py_satellite", "py_semantic"]):
    visualize_rgb_image(dataset_dict[key], index=0, title=f"{key}: {type(rasterizer_dict[key]).__name__}", ax=axes[i])
fig.show()

### 3.6.2 <a id='3-6-2'>Visualize Trajectory: Semantic View</a>

[Table of Contents](#0.1)

In [ ]:
cfg["raster_params"]["map_type"] = "py_semantic"

# raster object for visualization
rast = build_rasterizer(cfg, dm)

# EgoDataset object
dataset = EgoDataset(cfg, zarr_dataset, rast)

# select one example from our dataset
data = dataset[50]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])

# plot ground truth trajectory
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.rcParams['figure.figsize'] = 10, 10
plt.title('Ground Truth Trajectory of Autonomous Vehicle',fontsize=15)
plt.imshow(im[::-1])
plt.show()

### 3.6.3 <a id='3-6-3'>Visualize Trajectory: Satellite View</a>

[Table of Contents](#0.1)

We can get the satellite view by changing the parameter **map_type** inside **raster_params** in our configuration file **cfg**.

In [ ]:
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)

# EgoDataset object
dataset = EgoDataset(cfg, zarr_dataset, rast)
data = dataset[50]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.rcParams['figure.figsize'] = 10, 10
plt.title('Satellite View: Ground Truth Trajectory of Autonomous Vehicle',fontsize=15)
plt.imshow(im[::-1])
plt.show()

### 3.6.4 <a id='3-6-4'>Visualize Agent</a>

[Table of Contents](#0.1)

We will visualize our agents using AgentDataset class. 

In [ ]:
# AgentDataset object
dataset = AgentDataset(cfg, zarr_dataset, rast)
data = dataset[50]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.rcParams['figure.figsize'] = 10, 10
plt.title('Agent',fontsize=15)
plt.imshow(im[::-1])
plt.show()

The **green** box is our AV agent. The **blue** boxes are entities which we are captured by the sensors. We want to predict the motion of these entities so that our AV can more effectively predict its path. 

### 3.6.5 <a id='3-6-5'>Visualize Individual Scene: Semantic</a>

[Table of Contents](#0.1)

We will visualize the scene in depth.

In [ ]:
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_semantic"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))
    
# animation    
anim = animate_solution(images)
HTML(anim.to_jshtml())

* The green box is our AV agent and the arrow on top of it represent its motion. 
* The blue boxes are agents (cars, cyclists, predestrians).
* We can see intersection of roads.
* In animation we can see the AV is moving on straight path.

### 3.6.6 <a id='3-6-6'>Visualize Individual Scene: Satellite</a>

[Table of Contents](#0.1)

We are now going to visualize the satellite view for more detailed understanding.

In [ ]:
# satellite view
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))
    
# animation    
anim = animate_solution(images)
HTML(anim.to_jshtml())

* We can see the green AV agent and blue entities (cars, bicycles and pedestrians).
* We can see our agent in motion in realtion to the movement of other agents (vehicles).

In [ ]:
gc.collect()

# 4. <a id='4'>Pytorch Baseline🔦</a>

[Table of Contents](#0.1)

## 4.1 <a id='4-1'>Configuration</a>

Let us first make our configurations for training and testing then we will load our data.

In [ ]:
DEBUG = True

# training cfg
training_cfg = {
    
    'format_version': 4,
    
     ## Model options
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
    },

    ## Input raster parameters
    'raster_params': {
        
        'raster_size': [224, 224], # raster's spatial resolution [meters per pixel]: the size in the real world one pixel corresponds to.
        'pixel_size': [0.5, 0.5], # From 0 to 1 per axis, [0.5,0.5] would show the ego centered in the image.
        'ego_center': [0.25, 0.5],
        'map_type': "py_semantic",
        
        # the keys are relative to the dataset environment variable
        'satellite_map_key': "aerial_map/aerial_map.png",
        'semantic_map_key': "semantic_map/semantic_map.pb",
        'dataset_meta_key': "meta.json",

        # e.g. 0.0 include every obstacle, 0.5 show those obstacles with >0.5 probability of being
        # one of the classes we care about (cars, bikes, peds, etc.), >=1.0 filter all other agents.
        'filter_agents_threshold': 0.5
    },

    ## Data loader options
    'train_data_loader': {
        'key': "scenes/train.zarr",
        'batch_size': 12,
        'shuffle': True,
        'num_workers': 4
    },

    ## Train params
    'train_params': {
        'checkpoint_every_n_steps': 5000,
        'max_num_steps': 100 if DEBUG else 10000
    }
}

# inference cfg
inference_cfg = {
    
    'format_version': 4,
    'model_params': {
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [300, 300],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    
        'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 8,
        'shuffle': False,
        'num_workers': 4
    }

}

## 4.2 <a id='4-2'>Loading Training Data</a>

[Table of Contents](#0.1)

In [ ]:
# root directory
DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"

#submission
SINGLE_MODE_SUBMISSION = f"{DIR_INPUT}/single_mode_sample_submission.csv"
MULTI_MODE_SUBMISSION = f"{DIR_INPUT}/multi_mode_sample_submission.csv"

# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
print(training_cfg)

## 4.3 <a id='4-3'>Training DataLoader</a>

[Table of Contents](#0.1)


In [ ]:
# training cfg
train_cfg = training_cfg["train_data_loader"]

# rasterizer
rasterizer = build_rasterizer(training_cfg, dm)

# dataloader
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(training_cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
print(train_dataset)

## 4.4 <a id='4-4'>Model: resnet50 (Pytorch)</a>

[Table of Contents](#0.1)

This is for the purpose of demonstration only. You can set [pretrained=True]() and train your model in separate notebook keeping on internet connection for downloading pretrained weights and make inference in separate notebook keeping internet off since subission requires you to turn off your internet connection. I will use my trained weights for making inference.

In [ ]:
class LyftModel(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()
        
        # set pretrained=True while training
        self.backbone = resnet34(pretrained=False) 
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )
        
        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        backbone_out_features = 512
        
        # X, Y coords for the future positions (output shape: Bx50x2)
        num_targets = 2 * cfg["model_params"]["future_num_frames"]

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.logit = nn.Linear(4096, out_features=num_targets)
        
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        
        x = self.head(x)
        x = self.logit(x)
        
        return x

## 4.5 <a id='4-5'>Compilation</a>

[Table of Contents](#0.1)

In [ ]:
# compiling model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LyftModel(training_cfg).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction="none")

In [ ]:
# get hardware type (CPU, GPU, TPU)
device

## 4.6 <a id='4-6'>Training</a>

[Table of Contents](#0.1)

In [ ]:
# training loop
tr_it = iter(train_dataloader)
progress_bar = tqdm(range(training_cfg["train_params"]["max_num_steps"]))

losses_train = []

for _ in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    model.train()
    torch.set_grad_enabled(True)
    
    # forward pass
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    
    outputs = model(inputs).reshape(targets.shape)
    loss = criterion(outputs, targets)

    # not all the output steps are valid, but we can filter them out from the loss using availabilities
    loss = loss * target_availabilities
    loss = loss.mean()
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses_train.append(loss.item())
        
    progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

## 4.7 <a id='4-7'>Saving Model</a>

[Table of Contents](#0.1)

In [ ]:
# save full trained model
torch.save(model.state_dict(), f'model_state_last.pth')

# 5. <a id='5'>Prediction and Results📖</a>

[Table of Contents](#0.1)

Be careful you have turned of your internet connection in order to make submission. Please make separate notebook for the inference. This notebook is for the purpose of demonstration. Train using pretrained resnet weights in one notebook and make inference using another notebook.

Inference: Trained using Google Colab
* Model: Single mode baseline (resnet18)
* Steps: Trained for 30000 iterations (batch 16)
* Size: Input size 300px, history 1s (10 frames)
* Optimizer: Adam (1e-3)
* Loss: MSE Loss
* LB: 246.349

Inference: Trained using Google Colab
* Model: Single mode baseline (resnet18)
* Steps: Trained for 30000 iterations (batch 16)
* Size: Input size 350px, history 1s (10 frames)
* Optimizer: Adam (1e-3)
* Loss: MSE Loss
* LB: 169.83

## 5.1 <a name='5-1'>Test DataLoader</a>

[Table of Contents](#0.1)

In [ ]:
# test configuration
test_cfg = inference_cfg["test_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(inference_cfg, dm)

# Test dataset/dataloader
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(inference_cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])


print(test_dataloader)

## 5.2 <a id='5-2'>Getting Predictions</a>

[Table of Contents](#0.1)

In [ ]:
# Saved state dict from the training notebook
WEIGHT_FILE = '/kaggle/input/lyft-l5-weights/model_state_last.pth'
model_state = torch.load(WEIGHT_FILE, map_location=device)
model.load_state_dict(model_state)

**For loading checkpoint**

WEIGHT_FILE = '/kaggle/input/lyft-l5-weights/resnet34_300x300_model_state_15000.pth'
<br>
model_state = torch.load(WEIGHT_FILE, map_location=device)
<br>
model.load_state_dict(model_state['model_state_dict'])

In [ ]:
device

We need to run below two cell to make predictions and generate **submission.csv**. I am going to use my submission.csv here since I am getting memory error while making predictions. 

In [ ]:
'''
model.eval()
torch.set_grad_enabled(False)

# store information for evaluation
future_coords_offsets_pd = []
timestamps = []

agent_ids = []
progress_bar = tqdm(test_dataloader)
for data in progress_bar:
    
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)

    outputs = model(inputs).reshape(targets.shape)
    
    future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
    timestamps.append(data["timestamp"].numpy().copy())
    agent_ids.append(data["track_id"].numpy().copy())
'''

## 5.3 <a id='5-3'>Submission</a>

[Table of Contents](#0.1)

In [ ]:
# submission.csv
'''
write_pred_csv('submission.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
              )
              
'''
model_sub = pd.read_csv('/kaggle/input/lyft-l5-inference-batch64-resnet18/submission.csv')
model_sub.to_csv('submission.csv', index = False)

# 6. <a id='6'>References📎</a>

[Table of Contents](#0.1)

* https://www.twi-global.com/technical-knowledge/faqs/what-is-an-autonomous-vehicle
* [Negative log-likelihood](https://github.com/lyft/l5kit/blob/master/competition.md)
* https://www.kaggle.com/pestipeti/pytorch-baseline-train
* https://self-driving.lyft.com/level5/prediction/
* https://www.kaggle.com/corochann/lyft-deep-into-the-l5kit-library
* https://www.kaggle.com/corochann/lyft-deep-into-the-l5kit-library#1.-Understanding-Rasterizer-class
* https://www.kaggle.com/gpreda/lyft-first-data-exploration